In [1]:
import os

In [2]:
import pandas as pd
import time
import pickle

In [3]:
import matgl
from matgl.ext.ase import PESCalculator, MolecularDynamics, Relaxer

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from disalloy.cellfunc import jitter_ia, jitter_ic, compute_distortion
from disalloy.grid_run import run_packet

In [5]:
pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")

/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_std", torch.tensor(da

In [6]:
relaxer_2021_2_8_PES = Relaxer(pot)

In [7]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [8]:

# def run_relaxer(structure, relaxer):
#     b = relaxer(structure.copy())
#     atoms_clean = b['trajectory'].atoms.copy()
#     atoms_clean.calculator = None
#     return atoms_clean, b['trajectory'].energies[-1]
#     # return b['trajectory'].atoms.cell.array,
#     # b['trajectory'].atoms.get_positions(), 
#     # b['trajectory'].energies[-1]

def relaxer_direct_2021_2_8_PES(row):
    structure = row['init_structure'].copy()
    b = relaxer_2021_2_8_PES.relax(structure)
    atoms_clean = b['trajectory'].atoms.copy()
    atoms_clean.calculator = None
    return atoms_clean, b['trajectory'].energies[-1]
def relaxer_jitter_a_01_2021_2_8_PES(row, ia = 0.01):
    structure = row['init_structure'].copy()
    structure = jitter_ia(structure, ia)
    b = relaxer_2021_2_8_PES.relax(structure)
    atoms_clean = b['trajectory'].atoms.copy()
    atoms_clean.calculator = None
    return atoms_clean, b['trajectory'].energies[-1]
def relaxer_jitter_c_005_2021_2_8_PES(row, ic = 0.01):
    structure = row['init_structure'].copy()
    structure = jitter_ic(structure, ic)
    b = relaxer_2021_2_8_PES.relax(structure)
    atoms_clean = b['trajectory'].atoms.copy()
    atoms_clean.calculator = None
    return atoms_clean, b['trajectory'].energies[-1]

## creating a dabase 'df_calc' and running the direct relaxation in M3GNet model

In [9]:
from disalloy.grid_run import run_packet

In [10]:
df = structure_DB.copy()
# if os.path.exists('df_calc.pkl'):
#     df = pd.read_pickle('df_calc.pkl')

In [11]:
relaxer_dic_2021_2_8_PES = {'calculator_label': 'relaxer_dic_2021_2_8_PES',
                            'relaxer': relaxer_direct_2021_2_8_PES}

In [12]:
print(df.attrs)

{'number_of_structures': {1: 8, 2: 56, 3: 336, 4: 4046, 5: 15904, 6: 17080, 7: 23968, 8: 128540, 9: 140000, 10: 180000}, 'is_exhaustive': {1: True, 2: True, 3: True, 4: True, 5: True, 6: False, 7: False, 8: False, 9: False, 10: False}}


In [13]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 10, 1)
print('{}s'.format(time.time() - ti))

2024-09-04 01:52:39 - starting packet run: attempting 10 runs of size 1

2024-09-04 01:53:52 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-04 01:53:52 - Succesfuly ran packet of 8 runs in 33.562543869018555s

73.45378255844116s


In [14]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 20, 2)
print('{}s'.format(time.time() - ti))

2024-09-04 01:53:52 - starting packet run: attempting 20 runs of size 2

2024-09-04 01:57:15 - index of ran strctures:
 [62, 57, 10, 27, 43, 8, 55, 54, 49, 56, 61, 58, 42, 51, 15, 29, 39, 46, 32, 9]

2024-09-04 01:57:15 - Succesfuly ran packet of 20 runs in 169.18237948417664s

203.40549325942993s


In [15]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 10, 3)
print('{}s'.format(time.time() - ti))

2024-09-04 01:57:15 - starting packet run: attempting 10 runs of size 3

2024-09-04 01:59:02 - index of ran strctures:
 [350, 398, 185, 286, 276, 282, 284, 95, 315, 65]

2024-09-04 01:59:02 - Succesfuly ran packet of 10 runs in 82.15681147575378s

107.04184341430664s


## now we run with another relaxer (using cell relaxer), but we calculate the same indeces previously calculated

In [17]:
relaxer_dic_jitter_c_005_2021_2_8_PES = {
    'calculator_label': 'relaxer_dic_jitter_c_005_2021_2_8_PES',
    'relaxer': relaxer_jitter_c_005_2021_2_8_PES
}

In [18]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_jitter_c_005_2021_2_8_PES, 10, 1, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-04 01:59:03 - starting packet run: attempting 10 runs of size 1

2024-09-04 02:00:20 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-04 02:00:20 - Succesfuly ran packet of 8 runs in 45.17092323303223s

76.77451825141907s


In [19]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_jitter_c_005_2021_2_8_PES, 20, 2, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-04 02:00:20 - starting packet run: attempting 20 runs of size 2

2024-09-04 02:03:48 - index of ran strctures:
 [8, 9, 10, 15, 27, 29, 32, 39, 42, 43, 46, 49, 51, 54, 55, 56, 57, 58, 61, 62]

2024-09-04 02:03:48 - Succesfuly ran packet of 20 runs in 177.89048886299133s

207.35061311721802s


In [20]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_jitter_c_005_2021_2_8_PES, 10, 3, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-04 02:03:48 - starting packet run: attempting 10 runs of size 3

2024-09-04 02:06:18 - index of ran strctures:
 [65, 350, 398, 276, 185, 282, 315, 284, 286, 95]

2024-09-04 02:06:18 - Succesfuly ran packet of 10 runs in 86.7529239654541s

150.13846492767334s


In [21]:
calculated_index = df[df[relaxer_dic_2021_2_8_PES['calculator_label']].notnull()].index


In [22]:
for i, j, k  in zip(df.loc[calculated_index, 'init_structure'], 
                df.loc[calculated_index, 'relaxer_dic_2021_2_8_PES'].array, 
             df.loc[calculated_index, 'relaxer_dic_jitter_c_005_2021_2_8_PES'].array):
    print(compute_distortion(i, j[0]))
    print(compute_distortion(i, k[0]))
    print()

1.5728799997364826e-06
0.008478383019435431

3.4711914702793634e-06
0.014992121608954932

1.869416926911698e-05
0.0055260174383238575

6.591054889187993e-06
0.0013269749304700654

2.8925424054246692e-05
0.041815786530901525

1.5884375891685432e-05
0.016760385825690095

2.5211789102510017e-06
0.002978565993329864

2.0687255793244462e-05
0.2675571670462636

0.16736472938249838
0.1556391981118748

0.0033938860022789045
0.005400236771320762

0.08171670991394009
0.08455231601816081

0.001232368835267994
0.006085425992822564

0.10549966456578831
0.10507149580550249

0.02025507197885123
0.022524359721694635

0.0801752265765604
0.08018755315184305

0.27751017164710384
0.2743531062992893

0.15023570149026255
0.14603844484541006

0.04627730708397882
0.04563094194938786

0.28299233544182945
0.2952608167347834

0.148931955963863
0.13934529862433764

0.2906526056232204
0.2871462242578758

0.15756575302427603
0.15675519381732916

0.16465976636159527
0.1634978212083279

0.029787477574621146
0.0323071